In [ ]:
%matplotlib inline

# The Sobieski's SSBJ MDO problem


In [ ]:
from __future__ import annotations

from gemseo import configure_logger
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.problems.mdo.sobieski.core.problem import SobieskiProblem
from gemseo.problems.mdo.sobieski.disciplines import SobieskiAerodynamics
from gemseo.problems.mdo.sobieski.disciplines import SobieskiMission
from gemseo.problems.mdo.sobieski.disciplines import SobieskiPropulsion
from gemseo.problems.mdo.sobieski.disciplines import SobieskiStructure

from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario

configure_logger()

Firstly,
we instantiate the discipline of Sobieski's SSBJ problem:



In [ ]:
mission = SobieskiMission()
structure = SobieskiStructure()
propulsion = SobieskiPropulsion()
aerodynamics = SobieskiAerodynamics()

as well as the design space:



In [ ]:
design_space = SobieskiProblem().design_space

Secondly,
we define the uncertain space:



In [ ]:
uncertain_space = ParameterSpace()

with an uncertainty over the constant `"c_4"`:



In [ ]:
uncertain_space.add_random_variable(
    "c_4", "OTNormalDistribution", mu=0.01375, sigma=0.01375 * 0.05
)

and an uncertainty over the design variable `"x_2"`,
expressed as an additive term `"u_x_2"`
defined just after in the [UMDOScenario][gemseo_umdo.scenarios.umdo_scenario.UMDOScenario]:



In [ ]:
uncertain_space.add_random_variable(
    "u_x_2", "OTNormalDistribution", mu=0.0, sigma=1 * 0.05
)

Then,
we build an [UMDOScenario][gemseo_umdo.scenarios.umdo_scenario.UMDOScenario]
to maximize a first-order linearization-based estimation (default estimation method)
of the expectation $\mathbb{E}[y_4]$:



In [ ]:
scenario = UMDOScenario(
    [mission, structure, propulsion, aerodynamics],
    "y_4",
    design_space,
    uncertain_space,
    "Mean",
    formulation_name="MDF",
    statistic_estimation="TaylorPolynomial",
    # statistic_estimation_parameters={"second_order": True},
    maximize_objective=True,
    uncertain_design_variables={"x_2": "{}+u_x_2"},
)

while satisfying margin constraints
of the form $\mathbb{E}[g_i]+3\mathbb{S}[g_i]$



In [ ]:
scenario.add_constraint("g_1", "Margin", factor=3.0)
scenario.add_constraint("g_2", "Margin", factor=3.0)
scenario.add_constraint("g_3", "Margin", factor=3.0)

and execute it with a gradient-free optimizer:



In [ ]:
scenario.execute(algo_name="NLOPT_COBYLA", max_iter=100)

Lastly,
we can plot the optimization history view:



In [ ]:
scenario.post_process(post_name="OptHistoryView", save=False, show=True)